# Libraries

In [2]:
import pandas as pd
import numpy as np
import os

# Import Data

01. Import data

In [6]:
# main folder
path = r'C:\Users\steve\Documents\11.24 Instacart Basket Analysis'

# products csv
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

# wrangled order csv
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)


02. Create test data to practice on

In [8]:
# Create a dataframe
df_test = pd.DataFrame()

# Create a mixed type column
df_test['mix'] = ['a', 'b', 1, True]

# Data Consistency Prep

## df_prods

In [13]:
df_prods.describe()

# all id columns can be updated to string data type

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


In [15]:
price_high_outliers = df_prods[df_prods['prices'] >= 25]

price_high_outliers

# Row 21554 & 33664 look like they have errors for their prices

,product_id,product_name,aisle_id,department_id,prices
9020,9020,Boneless Skinless Chicken Thighs,35,12,25.0
19392,19391,Turkey Breast Tenderloins,49,12,25.0
21468,21467,Wild Caught Raw Shrimp,15,12,25.0
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
25580,25579,Naturally Smoked Trout Fillet,15,12,25.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0
40490,40486,Chicken Tenders,49,12,25.0


In [17]:
df_prods.shape

(49693, 5)

In [19]:
# Delete 2 outliers
df_prods.drop([21554, 33666], inplace = True)

In [21]:
df_prods.shape

(49691, 5)

01. Mixed-Type Data

In [24]:
# check if df_test has any mixed-type columns
for col in df_test.columns.tolist():
    weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_test[weird]) > 0:
        print(col)

mix


In [26]:
# fix mixed data type found by settng the 'mix' column to string
df_test['mix'] = df_test['mix'].astype('str')

02. Missing Values

In [29]:
# returns number of missing data by column
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [31]:
# creates new df and stores the missing values in it
df_nan = df_prods[df_prods['product_name'].isnull() == True]
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [33]:
# run the shape function to compare the number of missing rows to overall rows
df_prods.shape

(49691, 5)

In [35]:
# create a new df with only rows that have no missing data i.e. "remove" missing values
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]
# run shape to check that number of rows have decreased, thus successfully removing values
df_prods_clean.shape

(49675, 5)

03. Duplicates

In [38]:
# creates a subset containing only the full duplicates from df_prods_clean
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [40]:
# create df without the duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()
df_prods_clean_no_dups.shape

(49670, 5)

## df_ords

In [43]:
# (QUESTION 2) 
df_ords.head()
# df_ords.tail()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [45]:
# (QUESTION 2)
df_ords.describe()

# The Unnamed column should be deleted - it looks like the index column was duplicated
# order_id, user_id, and order_number should be changed to data type string

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


01. Mixed-Type Data

In [48]:
# (QUESTION 3 & 4)
# check if df_ords has any mixed-type columns
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print(col)
    else:
        print(col, 'not a mixed-type')

# There are no mixed-type data columns in df_ords

Unnamed: 0 not a mixed-type
order_id not a mixed-type
user_id not a mixed-type
order_number not a mixed-type
orders_day_of_week not a mixed-type
order_hour_of_day not a mixed-type
days_since_prior_order not a mixed-type


02. Missing Values

In [50]:
# (QUESTION 5 & 6)
# returns number of missing data by column in df_ords
df_ords.isnull().sum()

# There are 206209 missing values in the days_since_prior order column.
# These could indicate 1 time customers who, therefore, have not had a prior order.
# I will leave these cells as is since the missing values indicates a 1 time customer.

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

03. Duplicates

In [54]:
# (QUESTION 7)
# creates a subset containing only the full duplicates from df_ords
df_ords_dups = df_ords[df_ords.duplicated()]
df_ords_dups

# (QUESTION 8)
# There are no duplicates, no action taken.

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


In [56]:
df_prods_clean_no_dups.shape

(49670, 5)

In [58]:
df_ords.shape

(3421083, 7)

# Export Data

In [116]:
# export cleaned prods
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

In [117]:
# (QUESTION 9)
# export cleaned prods
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'))